<a href="https://colab.research.google.com/github/facuarre96/chlorine-lm-classifier/blob/main/Research_scrapping_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#set up + import
!pip install pandas openpyxl requests beautifulsoup4
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [ ]:
#more set up + import
from google.colab import drive
drive.mount('/content/drive')

file_path2 = '/content/drive/MyDrive/1-s2.0-S027323002200174X-mmc2.xlsx'
df1 = pd.read_excel(file_path2)

file_path3 = '/content/drive/MyDrive/1-s2.0-S027323002200174X-mmc3.xlsx'
df2 = pd.read_excel(file_path3)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# test acsess
print(df1.head())
print(df2.head())

                                         short_title  \
0  Evaluation of water washes for the removal of ...   
1  Effect of sodium sulfite on the mutagenicity o...   
2  Measuring the genotoxic potential in two drink...   
3  Identification and quantification of the mutag...   
4  Erlotinib-related rhabdomyolysis: The role of ...   

                                            abstract       Label  \
0  The effectiveness of distd. water and water co...  Irrelevant   
1                                                NaN  Irrelevant   
2                                                NaN    Relevant   
3                                                NaN  Irrelevant   
4                                                NaN  Irrelevant   

            Subgroup                           url      PMID PMCID  \
0                NaN  10.1007/978-1-4419-9090-7_23  15174591   NaN   
1                NaN            10.1007/BF00200793   1392318   NaN   
2  NonClinical Water            10.1007/BF00

In [ ]:
#check for value of abstract task.
column_name = 'abstract'
#get total missing
missing_values_count_mmc2 = df1[column_name].isna().sum()
print("missing value count for CHE:" , missing_values_count_mmc2)

all_count_mmc2 =  len(df1)
print('total rows:', all_count_mmc2)

print('ratio:' ,(missing_values_count_mmc2 / all_count_mmc2) * 100)


missing_values_count_mmc3 = df2[column_name].isna().sum()
print('missing value count for CHS:' ,missing_values_count_mmc3)

all_count_mmc3 =  len(df2)
print('total rows:', all_count_mmc2)

print('ratio:' ,(missing_values_count_mmc3 / all_count_mmc3) * 100)

missing value count for CHE: 376
total rows: 9788
ratio: 3.8414384961176955
missing value count for CHS: 1036
total rows: 9788
ratio: 10.203880626415836


In [ ]:
url = 'https://doi.org/10.1007/978-1-4419-9090-7_23'
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    page_content = response.content
    print('works')
else:
    print('Failed. Status code:' , response.status_code)

works


In [ ]:
url = 'https://doi.org/10.1023/a:1016057222128'


response = requests.get(url)

# Check request success
if response.status_code == 200:
    # Parse using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    abstract = None

    abstract_div = soup.find('div', class_='Abstract')
    if abstract_div:
        abstract = abstract_div.get_text(strip=True)

    if not abstract:
        abstract_meta = soup.find('meta', {'name': 'description'})
        if abstract_meta:
            abstract = abstract_meta.get('content', None)
    if abstract:
        print('Abstract:' ,  abstract)
    else:
        print('Abstract not found in the web page.')
else:
    print('Failed to retrieve the web page. Status code:' , response.status_code)

Abstract not found in the web page.


In [ ]:
#get HTML of url
def get_html(url):
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            return response.text
        else:
            print(f"Failed to retrieve content from {url}. Status code: {response.status_code}")
            return None
    except Exception as e:
        print(f"Error accessing {url}: {e}")
        return None


url = "https://doi.org/10.1016/j.det.2010.03.013"
html_content = get_html(url)
print(html_content)











<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "https://www.w3.org/TR/html4/loose.dtd">
<html>
<head>
<meta charset="utf-8">
<META http-equiv=Content-Type content="text/html">
<meta HTTP-EQUIV="REFRESH" content="2; url='/retrieve/articleSelectSinglePerm?Redirect=https%3A%2F%2Fwww.sciencedirect.com%2Fscience%2Farticle%2Fpii%2FS0733863510000549%3Fvia%253Dihub&amp;key=b71f1c43f609d14202a640be09de2a154b2b614e'"/>
<script type="text/JavaScript" src="/retrieve/static/script/siteCatalystHelper.js"></script>
<script type="text/JavaScript" src="https://ajax.googleapis.com/ajax/libs/jquery/1.10.1/jquery.min.js"></script>
<script src="https://assets.adobedtm.com/4a848ae9611a/519188bfaff2/launch-d9c9ae460653.min.js"></script>
<title>Redirecting</title>
<META http-equiv=Content-Type content="text/html; charset=utf-8">
<script language=JavaScript type="text/javascript">
var timerStart = Date.now();
var pageName = "Article Locator Bypass";

if (pageName == "") {
	pageName = "

In [ ]:
valid_urls_df = df1[(df1['abstract'].isna() | (df1['abstract'] == '')) & df1['url'].notna()]

# Save time to do less rows
sampled_df = valid_urls_df.sample(n=83)

# fetch abstract from URL
def fetch_abstract(url):
    # URL needs DOI or not
    if not url.startswith('http'):
        url = 'https://doi.org/' + url

    try:
        response = requests.get(url, allow_redirects=True)
        response.raise_for_status()
        if response.headers['Content-Type'].startswith('application/pdf'):
            return "PDF Content"
        elif response.headers['Content-Type'].startswith('text/html'):
            # Extract text from HTML content
            soup = BeautifulSoup(response.text, 'html.parser')



            # check if redirect needed
            if "Redirecting" in soup.text:
                return "Redirecting"
            return soup.get_text()
        else:
            return "Other Content Type"
    except requests.exceptions.RequestException as e:
        return str(e)

# Iterate over the sampled rows
for index, row in sampled_df.iterrows():
    abstract = fetch_abstract(row['url'])
    sampled_df.at[index, 'abstract'] = abstract

# Output
sampled_df


,short_title,abstract,Label,Subgroup,url,PMID,PMCID,Match
72,Accelerator analyses of tributyltin chloride a...,Redirecting,Irrelevant,NaN,10.1016/j.marpolbul.2004.01.002,15041438,NaN,title - 0.993
22,APIC guidelines for handwashing and hand antis...,Redirecting,Irrelevant,NaN,10.1016/0196-6553(95)90070-5,7503437,NaN,title - 0.993
64,Point-of-care controls for nosocomial legionel...,Redirecting,Irrelevant,NaN,10.1016/j.jhin.2005.11.004,16621143,NaN,title - 0.989
66,Use of sodium dichloroisocyanurate for floor d...,Redirecting,Irrelevant,NaN,10.1016/j.jhin.2009.02.017,19376611,NaN,title - 0.983
67,Hospital cleaning: problems with steam cleanin...,Redirecting,Irrelevant,NaN,10.1016/j.jhin.2009.04.009,19540620,NaN,title - 0.992
...,...,...,...,...,...,...,...,...
122,"""Effectiveness of electrolyzed acidic water in...",Redirecting,Irrelevant,NaN,10.4315/0362-028x-66.9.1540,14503702,NaN,title - 0.999
6,Case files of the University of California San...,\n\n\n\n\n\n\n\n\n\nCase Files of the Universi...,Irrelevant,NaN,10.1007/s13181-013-0309-8,23719961,NaN,title - 0.998
91,Myasthenic crisis presenting as isolated vocal...,Redirecting,Irrelevant,NaN,10.1016/s0735-6757(00)90031-7,10750943,NaN,title - 0.986
7,Enzymes and associated electron transport syst...,Redirecting,Irrelevant,NaN,10.1016/0005-2728(95)00092-5,8534676,NaN,title - 0.997


In [ ]:
#check how sucsessful scapping code is
count = 0

for index, row in sampled_df.iterrows():
    abstract = row['abstract']
    if "Client Error" not in abstract and "Redirecting" not in abstract:
        count += 1
        df1.at[index, 'abstract'] = abstract

print(count)
redirecting_count = sampled_df['abstract'].str.contains('Redirecting', na=False).sum()
print('potential:',  redirecting_count)
df1


12
potential: 53


,short_title,abstract,Label,Subgroup,url,PMID,PMCID,Match
0,Evaluation of water washes for the removal of ...,The effectiveness of distd. water and water co...,Irrelevant,NaN,10.1007/978-1-4419-9090-7_23,15174591,NaN,title - 0.996
1,Effect of sodium sulfite on the mutagenicity o...,\n\n\n\n\n\n\n\n\n\nEffect of sodium sulfite o...,Irrelevant,NaN,10.1007/BF00200793,1392318,NaN,title - 0.994
2,Measuring the genotoxic potential in two drink...,\n\n\n\n\n\n\n\n\n\nMeasuring the genotoxic po...,Relevant,NonClinical Water,10.1007/BF00201751,8219588,NaN,title - 0.996
3,Identification and quantification of the mutag...,\n\n\n\n\n\n\n\n\n\nIdentification and Quantif...,Irrelevant,NaN,10.1007/s001289900539,9323219,NaN,title - 0.992
4,Erlotinib-related rhabdomyolysis: The role of ...,\n\n\n\n\n\n\n\n\n\nErlotinib-related rhabdomy...,Irrelevant,NaN,10.1007/s00280-015-2885-6,26482820,NaN,title - 0.994
...,...,...,...,...,...,...,...,...
9783,"2,3,7,8-Tetrachlorodibenzo-p-dioxin: molecular...",NaN,Irrelevant,NaN,NaN,NaN,NaN,NaN
9784,P7.01 Health Care Assistant (HCA)'s Compliance...,NaN,Irrelevant,NaN,NaN,NaN,NaN,NaN
9785,Investigation on factors influencing ultrasoun...,NaN,Irrelevant,NaN,NaN,NaN,NaN,NaN
9786,Preventing Clostridium Difficile Infection in ...,NaN,Irrelevant,NaN,NaN,NaN,NaN,title - 0.809


In [ ]:
#new values
missing_values_count_mmc2 = df1[column_name].isna().sum()
print("missing value count for CHE:" , missing_values_count_mmc2)

all_count_mmc2 =  len(df1)
print('total rows:', all_count_mmc2)

missing value count for CHE: 364
total rows: 9788


In [ ]:
valid_urls_df = df2[(df2['abstract'].isna() | (df2['abstract'] == '')) & df2['url'].notna()]

# Save time to do less rows
sampled_df2 = valid_urls_df.sample(n=463)

# fetch abstract from URL
def fetch_abstract(url):
    # URL needs DOI or not
    if not url.startswith('http'):
        url = 'https://doi.org/' + url

    try:
        response = requests.get(url, allow_redirects=True)
        response.raise_for_status()
        if response.headers['Content-Type'].startswith('application/pdf'):
            return "PDF Content"
        elif response.headers['Content-Type'].startswith('text/html'):
            # Extract text from HTML content
            soup = BeautifulSoup(response.text, 'html.parser')



            # check if redirect needed
            if "Redirecting" in soup.text:
                return "Redirecting"
            return soup.get_text()
        else:
            return "Other Content Type"
    except requests.exceptions.RequestException as e:
        return str(e)

# Iterate over the sampled rows
for index, row in sampled_df2.iterrows():
    abstract = fetch_abstract(row['url'])
    sampled_df2.at[index, 'abstract'] = abstract

# Output
sampled_df2

,short_title,abstract,Label,Subgroup,url,PMID,PMCID,Match
1219,Catalytic properties of an unspecific carboxyl...,403 Client Error: Forbidden for url: https://f...,Irrelevant,NaN,10.1111/j.1432-1033.1973.tb02892.x,4738938,NaN,title - 0.992
3552,Pharmacokinetics of trichloroethanol and metab...,Redirecting,Irrelevant,NaN,10.1002/jps.2600620404,4698974,NaN,title - 0.993
7992,Control of 36C1 uptake by isolated salivary gl...,Redirecting,Irrelevant,NaN,10.1016/0022-1910(74)90206-6,4412631,NaN,title - 0.994
4493,The metabolic fate of (vinyl-i-14C)dichlorvos ...,403 Client Error: Forbidden for url: https://w...,Irrelevant,NaN,10.3109/00498257109112270,5173023,NaN,title - 0.996
4734,Antifertility effects and metabolism of alpha ...,\n\n\nAntifertility Effects and Metabolism of ...,Irrelevant,NaN,10.1038/224083a0,5822916,NaN,title - 0.993
...,...,...,...,...,...,...,...,...
7617,Studies on Kidney Function: The Excretion of U...,403 Client Error: Forbidden for url: https://p...,Irrelevant,NaN,10.1042/bj0200461,16743680,NaN,title - 0.998
1451,GABA uptake in rat brain slices: inhibition by...,403 Client Error: Forbidden for url: https://o...,Irrelevant,NaN,10.1111/j.1471-4159.1973.tb12131.x,4698282,NaN,title - 0.996
4518,Urinary excretion of total trichloro-compounds...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,Irrelevant,NaN,10.1136/oem.29.3.328,5044605,NaN,title - 0.997
1736,Time and dose dependence of 3-methylcholanthre...,Redirecting,Irrelevant,NaN,10.1016/0006-2952(82)90606-2,7181960,NaN,title - 0.997


In [ ]:
#check how sucsessful scapping code is

count = 0

for index, row in sampled_df2.iterrows():
    abstract = row['abstract']
    if "Client Error" not in abstract and "Redirecting" not in abstract:
        count += 1
        df2.at[index, 'abstract'] = abstract

print(count)
redirecting_count = sampled_df2['abstract'].str.contains('Redirecting', na=False).sum()
print('potential:',  redirecting_count)
df2

99
potential: 205


,short_title,abstract,Label,Subgroup,url,PMID,PMCID,Match
0,Short term respiratory effects of acute exposu...,Acute exposure to Cl2 causes lung damage and r...,Relevant,Clinical,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,11351056,PMC1740147,NaN
1,Wheezing in the workplace: Occupational asthma...,It took almost a year for George Stephens to l...,Relevant,Clinical,https://www.sciencedirect.com/science/article/...,NaN,NaN,NaN
2,Hazard data bank: chlorine,This data sheet covers: description; uses; fir...,Relevant,Clinical,Internal Retrieval,NaN,NaN,NaN
3,Chemical safe practice pamphlet: chlorine,Practical information is provided under the he...,Relevant,Clinical,Internal Retrieval,NaN,NaN,NaN
4,Chlorine gas release associated with employee ...,"On June 27, 2011, a worker at a poultry proces...",Relevant,Clinical,Internal Retrieval,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
10148,Multielement Analysis of Soft Drinks by X-ray ...,Several com. soft drinks and resp. plastic bot...,Irrelevant,NaN,Internal Retrieval,NaN,NaN,NaN
10149,Study of removal effect on Mesocyclops leukart...,Cyclops of zooplankton propagates prolifically...,Irrelevant,NaN,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN,NaN,NaN
10150,Bench and full scale study of removal effect a...,Mesocyclops Leukarti of zooplankton propagates...,Irrelevant,NaN,Internal Retrieval,NaN,NaN,NaN
10151,QSAR models for predicting in vivo aquatic tox...,Quantitative structure-activity relationship (...,Irrelevant,NaN,Internal Retrieval,NaN,NaN,NaN


In [ ]:
#new values
missing_values_count_mmc3 = df2[column_name].isna().sum()
print("missing value count for CHE:" , missing_values_count_mmc3)

all_count_mmc3 =  len(df2)
print('total rows:', all_count_mmc3)

missing value count for CHE: 924
total rows: 10153


In [ ]:
empty_both_count = df1[(df1['url'].isna() | (df1['url'] == '')) & (df1['abstract'].isna() | (df1['abstract'] == ''))].shape[0]
print(empty_both_count)

empty_both_count = df2[(df2['url'].isna() | (df2['url'] == '')) & (df2['abstract'].isna() | (df2['abstract'] == ''))].shape[0]
print(empty_both_count)

293
573
